In [2]:
import pandas as pd
import torch
from torch.autograd import Variable
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
tracknames =['alpine-1.csv','aalborg.csv','f-speedway.csv']
datadictin = {}
datadictout = {}
framesin =[]
framesout=[]
for i in tracknames:
    path = os.path.join('train_data', i)
    print(path)
    datadictin[i] = pd.read_csv(path, index_col=False).iloc[:-1, 3:]
    datadictout[i] = pd.read_csv(path, index_col=False).iloc[:-1, 0:3]

#Concatenate all datasets
for i in tracknames:
    framesin.append(datadictin[i])
    framesout.append(datadictout[i])
    
inp = pd.concat(framesin) 
outp = pd.concat(framesout) 

#Drop missing values
inp.replace('', np.nan, inplace=True)
inp.dropna(inplace=True)
outp.replace('', np.nan, inplace=True)
outp.dropna(inplace=True)
outp.iloc[:,]

train_data/alpine-1.csv


FileNotFoundError: File b'train_data/alpine-1.csv' does not exist

In [ ]:
colsin = inp.shape[1]
colsout = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,colsin,colsout)

In [ ]:
# Get data from pandas in a Tensor Variable
x = Variable(torch.from_numpy(inp.iloc[:,:].as_matrix()).float(), requires_grad=True)
y = Variable(torch.from_numpy(outp.iloc[:,:].as_matrix()).float())

De onderstaande klasse definieert de RNN (in LSTM vorm). 

- De output van de LSTM wordt dmv een linear layer naar 3 output dimensions gemapt.
- De staat van de hidden layers worden bewaard zodat deze in de forward pass gebruikt kunnen worden.
- In de forward pass output de lstm een tuple met de output van de lstm en de staat van de hidden layer.
- TODO: Onderstaande is geen TwoLayerNet meer

In [ ]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.hidden_dim = H
        
        self.recurrent = nn.LSTM(D_in, H)
        self.linear = nn.Linear(H, D_out)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        """
        Before we've done anything, we dont have any hidden state.
        Refer to the Pytorch documentation to see exactly
        why they have this dimensionality.
        The axes semantics are (num_layers, minibatch_size, hidden_dim)
        """
        return (torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
        
    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        lstm_out, self.hidden = self.recurrent(x, self.hidden)
        y = self.linear(lstm_out)
        y_pred = F.tanh(y)
        
        return y_pred

Training van Recurrent Neural Network

Hieronder traint de RNN die hierboven beschreven staat. 

- Parameters moeten nog geoptimaliseerd worden.
- De error function is arbitrair, vereist een theoretisch argument.
- Elke cyclus duurt lang. 
- De output is in tanh vorm.

In [4]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 22, colsin, 20, colsout

model = TwoLayerNet(D_in, H, D_out)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
learning_rate = 1e-4

loss_fn = torch.nn.MSELoss(size_average=True)
optimizer = optim.SGD(model.parameters(), learning_rate, momentum=0.9)

for t in range(5):
    model.zero_grad()
    model.hidden = model.init_hidden()
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Variable of input data to the Module and it produces
    # a Variable of output data.
    y_pred = model(x)
    
    # Compute and print loss. We pass Variables containing the predicted and true
    # values of y, and the loss function returns a Variable containing the
    # loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])

    # Zero the gradients before running the backward pass.
    #model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Variables with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    

    # Update the weights using gradient descent. Each parameter is a Variable, so
    # we can access its data and gradients like we did before.
    optimizer.step()
    
# Test:
# Voorspellen lukt nog niet
# print(model(x[1]))

NameError: name 'colsin' is not defined

In [5]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 100, colsin, 20, colsout

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Variables for its weight and bias.

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LSTM(D_in, H),
    torch.nn.Linear(H, D_out),
    torch.nn.Sigmoid()
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.

learning_rate = 1e-4

loss_fn = torch.nn.MSELoss(size_average=True)
optimizer = optim.SGD(model.parameters(), learning_rate, momentum=0.9)

for t in range(100):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Variable of input data to the Module and it produces
    # a Variable of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Variables containing the predicted and true
    # values of y, and the loss function returns a Variable containing the
    # loss.
    loss = loss_fn(y_pred, y)

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Variables with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    

    # Update the weights using gradient descent. Each parameter is a Variable, so
    # we can access its data and gradients like we did before.
    optimizer.step()


NameError: name 'colsin' is not defined

In [149]:
save_path = 'trainedmodel_bbdata_500hidden_3out_21in_RNN'
# https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
# Save and load a custom model: https://discuss.pytorch.org/t/saving-custom-models/621/9
torch.save(model, save_path)

/home/jens/anaconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type TwoLayerNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [150]:
neural_net = torch.load('trainedmodel_bbdata_500hidden_3out_21in_RNN')

In [156]:
value= x[2000]

neural_net(value)

RuntimeError: invalid argument 2: dimension 1 out of range of 1D tensor at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/generic/THTensor.c:24